In [14]:
import pandas as pd
import numpy as np

In [15]:
filename = 'kosarak.dat.txt'

In [24]:
l = []
count = 0
with open(filename) as f:
    for line in f:
        l.append([int(ele) for ele in line.replace('\n','').split()])
        count+=1
        if count > 10000:
            break

In [25]:
df_data = pd.DataFrame(l)

In [26]:
freq_count = {}
for item in l:
    for ele in item:
        if ele in freq_count:
            freq_count[ele]+=1
        else:
            freq_count[ele] = 1

ordered_map = sorted(freq_count.items(), key=lambda x:x[1], reverse=True)

In [27]:
threshold = 0.01*count

In [30]:
i = 0
for item in ordered_map:
    if item[1]>=threshold:
        i+=1
    else:
        break
ordered_map = ordered_map[:i]

In [32]:
c1 = dict(ordered_map)

In [35]:
for item in l:
    tmp = [ele for ele in item if ele in c1 ]
    if len(tmp)>1:
        for i in range(len(tmp)-1):
            for j in range(i,len(tmp)):
                